In [2]:
# Mounting Drive to get file
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from pathlib import Path
import random

import numpy as np
import torch

%cd /content/drive/MyDrive/pinn-torch-master/
from model import Pinn
#from data import (
    # get_dataset,
#    get_orig_dataset,
#)
from trainer import Trainer

/content/drive/MyDrive/pinn-torch-master


In [4]:
from typing import List
from pathlib import Path
import random
import json

import numpy as np
import scipy
import torch
from torch.utils.data import Dataset


class PinnDataset(Dataset):
    def __init__(self, data: List[List[float]]):
        self.data = data
        self.examples = torch.tensor(
            data, dtype=torch.float32, requires_grad=True
        )

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        headers = ["t", "x", "y", "p", "u", "v"]
        return {key: self.examples[idx, i] for i, key in enumerate(headers)}


def load_jsonl(path, skip_first_lines: int = 0):
    with open(path, "r") as f:
        for _ in range(skip_first_lines):
            next(f)
        return [json.loads(line) for line in f]


def dump_json(path, data):
    with open(path, "w") as f:
        json.dump(data, f, indent=4)


def get_dataset(data_path: Path):
    data = load_jsonl(data_path, skip_first_lines=1)
    random.shuffle(data)

    # It's weird that the test data is a subset of train data, but
    # that's what the original paper does.
    split_idx = int(len(data) * 0.9)
    train_data = data
    test_data = data[split_idx:]

    # train_data = train_data[:10000]
    # train_data = train_data[:1000]

    min_x = min([d[1] for d in train_data])
    max_x = max([d[1] for d in train_data])

    train_data = PinnDataset(train_data)
    test_data = PinnDataset(test_data)
    return train_data, test_data, min_x, max_x


def get_orig_dataset():
    path = Path("./cylinder_nektar_wake.mat")
    data = scipy.io.loadmat(path)
    X_star = data["X_star"]  # N x 2
    x = X_star[:, 0:1]
    y = X_star[:, 1:2]

    U_star = data["U_star"]  # N x 2 x T
    P_star = data["p_star"]  # N x T
    print("P_Star:", P_star)
    t_star = data["t"]  # T x 1
    X_star = data["X_star"]  # N x 2

    N = X_star.shape[0]
    T = t_star.shape[0]

    # Rearrange Data
    XX = np.tile(X_star[:, 0:1], (1, T))  # N x T
    YY = np.tile(X_star[:, 1:2], (1, T))  # N x T
    TT = np.tile(t_star, (1, N)).T  # N x T

    UU = U_star[:, 0, :]  # N x T
    VV = U_star[:, 1, :]  # N x T
    PP = P_star  # N x T

    x = XX.flatten()[:, None]  # NT x 1
    y = YY.flatten()[:, None]  # NT x 1
    t = TT.flatten()[:, None]  # NT x 1

    u = UU.flatten()[:, None]  # NT x 1
    v = VV.flatten()[:, None]  # NT x 1
    p = PP.flatten()[:, None]  # NT x 1

    min_x = np.min(x)
    max_x = np.max(x)

    NOISE_SCALE = 0.1
    u += NOISE_SCALE * np.std(u) * np.random.randn(*u.shape)
    v += NOISE_SCALE * np.std(v) * np.random.randn(*v.shape)

    train_data = np.hstack((t, x, y, p, u, v))
    # Randomly sample 1000 points as test data
    #idx = np.random.choice(train_data.shape[0], 1000, replace=False)
    idx= np.arange(1000)
    print("idx", idx)
    test_data = train_data[idx, :]
    train_data = PinnDataset(train_data)
    test_data = PinnDataset(test_data)
    return train_data, test_data, min_x, max_x

In [5]:
train_data, test_data, min_x, max_x = get_orig_dataset()


P_Star: [[-0.10815457 -0.11115509 -0.1123679  ... -0.10035143 -0.09845376
  -0.09658143]
 [-0.10560371 -0.10881921 -0.1101648  ... -0.10137256 -0.09959008
  -0.09781616]
 [-0.10257633 -0.10599348 -0.10746686 ... -0.10204929 -0.10039933
  -0.09874095]
 ...
 [ 0.0066028   0.00272234  0.00081532 ...  0.01152894  0.01414437
   0.01651662]
 [ 0.007892    0.00402109  0.00209731 ...  0.00927242  0.01200894
   0.01451151]
 [ 0.00915729  0.00530504  0.00337089 ...  0.00691424  0.00975565
   0.01238328]]
idx [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 1

In [6]:
train_data, test_data, min_x, max_x = get_orig_dataset()

    # Model
    # hidden_dims = [128, 128, 128, 128, 128, 128, 128, 128]
model = Pinn(min_x, max_x)
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {num_params}")

    # Train
output_dir = Path(
        "result\pinn-large-tanh-bs128-lr0.005-lrstep1-lrgamma0.8-epoch20")
trainer = Trainer(model, output_dir)
#trainer.train(train_data)

    # Test
ckpt_dir = trainer.get_last_ckpt_dir()
trainer.load_ckpt(ckpt_dir)
outputs = trainer.predict(test_data)
lambda1 = trainer.model.lambda1.item()
lambda2 = trainer.model.lambda2.item()
print("lambda 1:", lambda1)
print("lambda 2:", lambda2)
loss = outputs["loss"]
preds = outputs["preds"]
print("Loss:", loss)
print("Preds:", preds)

P_Star: [[-0.10815457 -0.11115509 -0.1123679  ... -0.10035143 -0.09845376
  -0.09658143]
 [-0.10560371 -0.10881921 -0.1101648  ... -0.10137256 -0.09959008
  -0.09781616]
 [-0.10257633 -0.10599348 -0.10746686 ... -0.10204929 -0.10039933
  -0.09874095]
 ...
 [ 0.0066028   0.00272234  0.00081532 ...  0.01152894  0.01414437
   0.01651662]
 [ 0.007892    0.00402109  0.00209731 ...  0.00927242  0.01200894
   0.01451151]
 [ 0.00915729  0.00530504  0.00337089 ...  0.00691424  0.00975565
   0.01238328]]
idx [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 1

In [10]:
#print("Test Data U: ", test_data[0]["u"])
#print("Test Data V: ", test_data[0]["v"])
print("Test Data P: ", test_data[0]["p"])
print("Test Data P: ", test_data[1]["p"])
print("Test Data P: ", test_data[2]["p"])
print("preds:",preds[0][2])
print("preds:",preds[1][2].item())
print("preds:",preds[2][2].item())
for i in range(20):
  print("Test Data P: ", test_data[i]["p"].item())
  print("preds:",preds[i][2].item())

for i in range(20):
  print("Test Data U: ", test_data[i]["u"].item())
  print("preds:",preds[i][0].item())

for i in range(20):
  print("Test Data V: ", test_data[i]["v"].item())
  print("preds:",preds[i][1].item())


Test Data P:  tensor(-0.1082, grad_fn=<SelectBackward0>)
Test Data P:  tensor(-0.1112, grad_fn=<SelectBackward0>)
Test Data P:  tensor(-0.1124, grad_fn=<SelectBackward0>)
preds: tensor(0.0494, grad_fn=<SelectBackward0>)
preds: 0.0143937598913908
preds: 0.043848734349012375
Test Data P:  -0.10815457254648209
preds: 0.0494462326169014
Test Data P:  -0.11115509271621704
preds: 0.0143937598913908
Test Data P:  -0.1123678982257843
preds: 0.043848734349012375
Test Data P:  -0.11340326815843582
preds: 0.03591633588075638
Test Data P:  -0.11430054157972336
preds: 0.019974786788225174
Test Data P:  -0.11503268778324127
preds: 0.01586759462952614
Test Data P:  -0.1155279204249382
preds: 0.04665341228246689
Test Data P:  -0.11575125902891159
preds: 0.021356657147407532
Test Data P:  -0.11571624130010605
preds: 0.04416213184595108
Test Data P:  -0.11544903367757797
preds: 0.04323796182870865
Test Data P:  -0.11495650559663773
preds: 0.0387105755507946
Test Data P:  -0.11421672254800797
preds: 0.01

In [25]:
test_arr = np.array(test_data.data)
p = test_arr[:, 3]
u = test_arr[:, 4]
v = test_arr[:, 5]

p_pred = preds[:, 0]
u_pred = preds[:, 1]
v_pred = preds[:, 2]

for i in range(1000):
  if np.abs(u[i] - u_pred[i].item()) < 1:
    print("Test Data U: ", u[i])
    print("preds:",u_pred[i].item())

Test Data U:  1.0597989703807533
preds: 1.0781996250152588
Test Data U:  1.1244845056878054
preds: 1.2144402265548706
Test Data U:  1.1099826121283198
preds: 1.0362740755081177
Test Data U:  1.09108584515378
preds: 0.9124494791030884
Test Data U:  1.1051808399007632
preds: 0.837762713432312
Test Data U:  1.110772151735082
preds: 1.0451114177703857
Test Data U:  1.14231710342272
preds: 1.1191198825836182
Test Data U:  1.1025781695375008
preds: 1.1206949949264526
Test Data U:  1.1640930007005694
preds: 0.9238954186439514
Test Data U:  1.1548640909478793
preds: 1.0951472520828247
Test Data U:  1.1881061337006513
preds: 0.916983425617218
Test Data U:  1.1275858031508827
preds: 0.8570767045021057
Test Data U:  1.1616936947606245
preds: 1.1259558200836182
Test Data U:  1.1635815287327456
preds: 0.957371175289154
Test Data U:  1.1624655890495699
preds: 0.8645884394645691
Test Data U:  1.1540091109899946
preds: 1.0399563312530518
Test Data U:  1.1400386879848248
preds: 1.0188302993774414
Test 